In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
# 카메라의 면적 구하기

angle_degrees_1 = 57.5
angle_degrees_2 = 71.37796
h = 1100
angle_radians_1 = math.radians(angle_degrees_1)
angle_radians_2 = math.radians(angle_degrees_2)

x = 2 * h / math.tan(angle_radians_1)
print(x)

y = 2 * h / math.tan(angle_radians_2)
print(y)

# degree = math.atan2(1100,(x/2)) * 180 / math.pi
# print(degree)

S = x*y
S/(1920*1080)

In [2]:
def calculate_distance(point1, point2):
    # 두 점을 numpy 배열로 변환
    point1 = np.array(point1)
    point2 = np.array(point2)
    
    # 두 점 사이의 차이 계산
    difference = point2 - point1
    
    # 유클리드 거리 계산
    distance = np.linalg.norm(difference)
    
    return distance

# 두 점의 좌표 설정
point1 = (20408.03, 205.9421, 17528.64)  # 여기에 실제 좌표값을 넣으세요
point2 = (19274.9, 447.4, 17819.5)  # 여기에 실제 좌표값을 넣으세요

# 거리 계산
distance = calculate_distance(point1, point2)
print(distance)

1194.5229399104917


In [3]:
# 유니티상에서의 Vector3 거리를 계산
# 카메라 2 1.1km 위치: (25434,129.4599,31560.6504)
# 카메라 2 550 m 위치: (25434,129.4599,32110.4004)
# 카메라 3 275 m 위치: (25434,129.4599,32386.4004)
# 1100: 31561.3
# 1000: 31661
# 900: 31761
# 800: 31861.3
# 700: 31961.3
# 600: 32061.3
# 500: 32161.3
# 400: 32261.4 
# 300: 32361.4
# 200: 32461.47
# 100: 32561.62

point1 = (12887,764,30592)                  # 헬기
point2 = (11787,772.400024,30574.1094)      # 카메라
distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2 + (point2[2] - point1[2])**2)
print("두 점 사이의 거리:", distance)



두 점 사이의 거리: 1100.1775465676258


이미지 마스킹을 통해 비 영역을 제거한 픽셀 검출용 영상 제작

In [9]:
rain_image = cv2.imread("../../data/vtd/sky_rain/rain.png")
# snow_image = 
heli_image = cv2.imread("../../data/vtd/sky_rain/temp/0/09.png")
mask_image = cv2.cvtColor(rain_image,cv2.COLOR_BGR2RGB)
# lower_bound = np.array([70, 70, 70])
lower_bound = np.array([0, 0, 0])
upper_bound = np.array([110, 110, 110])
mask = cv2.inRange(mask_image, lower_bound, upper_bound)
heli_image[mask != 0] = [0, 0, 0]

cv2.imwrite('../../data/vtd/sky_rain/masked_heli_example.png', heli_image)
cv2.imshow('mask image', mask)
cv2.imshow('result image', heli_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# 마우스 클릭 이벤트에 반응하여 픽셀 RGB를 찍는 콜백 함수 정의
def show_rgb(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        b, g, r = image[y, x]
        print(f"RGB value at ({x}, {y}): ({r}, {g}, {b})")

image = cv2.imread("../../data/vtd/sky_rain/rain.png")
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', show_rgb)
while True:
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cv2.destroyAllWindows()

RGB value at (8, 176): (115, 113, 111)
RGB value at (8, 176): (115, 113, 111)
RGB value at (8, 176): (115, 113, 111)
RGB value at (17, 177): (115, 114, 113)
RGB value at (49, 181): (109, 108, 106)
RGB value at (80, 185): (115, 114, 113)
RGB value at (125, 193): (115, 113, 111)
RGB value at (150, 197): (115, 114, 113)
RGB value at (188, 201): (115, 114, 113)
RGB value at (208, 202): (115, 114, 113)
RGB value at (213, 202): (115, 114, 113)
RGB value at (216, 202): (115, 114, 113)
RGB value at (218, 202): (115, 114, 113)
RGB value at (218, 203): (115, 114, 113)
RGB value at (217, 205): (115, 114, 113)
RGB value at (216, 209): (115, 114, 113)
RGB value at (213, 213): (115, 114, 113)
RGB value at (207, 220): (115, 114, 113)
RGB value at (205, 221): (115, 114, 113)
RGB value at (200, 225): (104, 103, 102)
RGB value at (192, 228): (112, 111, 110)
RGB value at (184, 231): (115, 114, 113)
RGB value at (180, 232): (115, 114, 113)
RGB value at (172, 232): (115, 114, 113)
RGB value at (168, 232): 

In [28]:
# 반복문을 통해 한 폴더 내 모든 이미지파일에 대해 마스크를 적용
# 매트랩에서 ppm 검출 전 이 코드를 실행하여 
# temp 파일을 읽고 final 파일의 알맞은 고각 폴더에 파일을 옮긴다

rain_image = cv2.imread("../../data/vtd/sky_rain/rain.png")
mask_image = cv2.cvtColor(rain_image, cv2.COLOR_BGR2RGB)    # 마스크로 쓸 비가 촬영된 영상
lower_bound = np.array([0, 0, 0])
upper_bound = np.array([110, 110, 110])                     # 비의 RGB값을 검출할 경계 설정
mask = cv2.inRange(mask_image, lower_bound, upper_bound)
source_folder = "../../data/vtd/sky_rain/temp/180"
output_folder = "../../data/vtd/sky_rain/final"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

for i in range(19):
    filename = f"{i:02}.png"    # 00.png, 01.png, ..., 18.png 등의 이름 형식 이미지들에 대해 반복문 수행
    heli_image_path = os.path.join(source_folder, filename)
    heli_image = cv2.imread(heli_image_path)
    heli_image[mask != 0] = [0, 0, 0]
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, heli_image)

In [7]:
# 이미지 잘 불러와지는지 테스트용

# rain_image = cv2.imread("../../data/vtd/sky_rain/rain.png")
# if rain_image is None:
#     print("Error: Image not found or path is incorrect")
# else:
#     cv2.imshow('image', rain_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()